In [2]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from scraping import init_driver, to_excel

In [3]:
driver = init_driver()
driver.get("http://www.randhawa.us/games/retailer/nyu.html")


In [4]:
# extract

driver.find_element(By.ID, "practiceButton").click()
time.sleep(2) 

data = driver.find_element(By.ID, "result-table").text
splitdata = data.split("\n")
columns = splitdata[0].split(" ")
values_split = [row.split(" ") for row in splitdata[1:]]

table = pd.DataFrame(values_split, columns=columns[:4])

#### Exploring strategies

In [7]:
def strategy(table, choices, chosenchoices):
    week = max(table['Week'].astype(int))
    if week > 1 and len(chosenchoices) > week - 1:
        prev_choice = chosenchoices[week-1]
        sales_prev = int(table['Sales'][week-2])
        sales_curr = int(table['Sales'][week-1])
        if sales_prev != 0:
            lift = (sales_curr - sales_prev) / sales_prev
            lastchoice = max(chosenchoices)
            if lift < 0:
                print("Negative lift:", lift)
                return min(lastchoice+1, 3)
        else:
            print("Previous sales are zero, skipping division.")
    return 0



In [8]:
attempts = 100
all_data = []


for attempt in range(attempts):
    print(f"Attempt {attempt+1}/{attempts}")
    chosenchoices = [0]
    driver.find_element(By.ID, "practiceButton").click()
    time.sleep(1) 

    for i in range(15):
        data = driver.find_element(By.ID, "result-table").text
        splitdata = data.split("\n")
        columns = splitdata[0].split(" ")
        values_split = [row.split(" ") for row in splitdata[1:]]

        table = pd.DataFrame(values_split, columns=columns[:4])
        
        if len(table) == 0:
            print("Empty table encountered, breaking the loop.")
            break
        
        choice = strategy(table, [0, 1, 2, 3], chosenchoices)
        chosenchoices.append(choice)
        
        buttons = [
            driver.find_element(By.ID, "maintainButton"),
            driver.find_element(By.ID, "tenButton"),
            driver.find_element(By.ID, "twentyButton"),
            driver.find_element(By.ID, "fortyButton")
        ]
        buttons[choice].click()
        time.sleep(1) 

        if choice == 3:
            data = driver.find_element(By.ID, "result-table").text
            splitdata = data.split("\n")
            columns = splitdata[0].split(" ")
            values_split = [row.split(" ") for row in splitdata[1:]]

            table = pd.DataFrame(values_split, columns=columns[:4])
            chosenchoices.extend([0] * (13 - i))
            break

    foresight = driver.find_element(By.CSS_SELECTOR, "div[style*='position:relative;top:10px;left:6px;']").text
    table['foresight'] = foresight
    table['Choice'] = chosenchoices
    table['diff'] = float(table['foresight'][0].split(": ")[3].replace("%", ""))
    table['revenue'] = float(table['foresight'][0].split(", ")[0].split(": ")[1].replace("$", "").replace(",", ""))
    table['perfect'] = float(table['foresight'][0].split(", ")[1].split(": ")[1].replace("$", "").replace(",", ""))
    
    all_data.append(table)




Attempt 1/100
Negative lift: -0.14942528735632185
Negative lift: -0.6938775510204082
Negative lift: -0.42677824267782427
Attempt 2/100
Negative lift: -0.25316455696202533
Negative lift: -0.06976744186046512
Negative lift: -0.075
Attempt 3/100
Negative lift: -0.03125
Negative lift: -0.38461538461538464
Negative lift: -0.14754098360655737
Attempt 4/100
Negative lift: -0.09259259259259259
Negative lift: -0.21333333333333335
Negative lift: -0.15789473684210525
Attempt 5/100
Negative lift: -0.3291139240506329
Negative lift: -0.49056603773584906
Negative lift: -0.5137614678899083
Attempt 6/100
Negative lift: -0.3472222222222222
Negative lift: -0.0782608695652174
Negative lift: -0.6836158192090396
Attempt 7/100
Negative lift: -0.05
Negative lift: -0.4700460829493088
Negative lift: -0.14358974358974358
Attempt 8/100
Negative lift: -0.06593406593406594
Negative lift: -0.023529411764705882
Negative lift: -0.3504273504273504
Attempt 9/100
Negative lift: -0.4074074074074074
Negative lift: -0.69565

In [9]:
final_table = pd.concat(all_data, ignore_index=True)
to_excel(final_table, "scraping_results")

final_table

/Users/sheidamajidi/Desktop/Summer2024/MGSC670/G-HW1/retailer_game/game_scraping/scraping.py:33: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
Service process refused to terminate gracefully with SIGTERM, escalating to SIGKILL.
Traceback (most recent call last):
  File "/Users/sheidamajidi/anaconda3/lib/python3.11/site-packages/selenium/webdriver/common/service.py", line 172, in _terminate_process
    self.process.wait(60)
  File "/Users/sheidamajidi/anaconda3/lib/python3.11/subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sheidamajidi/anaconda3/lib/python3.11/subprocess.py", line 2038, in _wait
    raise TimeoutExpired(self.args, timeout)
subprocess.TimeoutExpired: Command '['/Users/sheidamajidi/.wdm/drivers/chromedriver/mac64/124.0.6367.207/chromedriver-mac-arm64/chromedriver', '--port=54784']' timed out after 60 secon

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect
0,1,60,87,1913,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752.0,101892.0
1,2,60,74,1839,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752.0,101892.0
2,3,54,147,1692,"Your revenue: $85,752, Perfect foresight strat...",1,15.8,85752.0,101892.0
3,4,54,45,1647,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752.0,101892.0
4,5,48,160,1487,"Your revenue: $85,752, Perfect foresight strat...",2,15.8,85752.0,101892.0
...,...,...,...,...,...,...,...,...,...
1495,11,36,100,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860.0,103596.0
1496,12,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860.0,103596.0
1497,13,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860.0,103596.0
1498,14,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860.0,103596.0
